In [ ]:
#Algo asi hay que hacer en la tarea
import cmath
import numpy as np
#Primero que nada necesitamos encontrar el valor de las matrices locales de las lineas, por lo que aquí se colocan los datos de la linea y despues se modelan por el modelo Pi para luego poder sumarlas al Y_buss
Ybuss = np.zeros((6, 6), dtype=complex)
Lista_datos_Lineas= [("1-2",0.031,0.126,0.062),
                     ("1-5",0.031,0.126,0.062),
                     ("2-3",0.063,0.252,0.122),
                     ("3-5",0.053,0.210,0.102),
                     ("3-6",0.084,0.236,0.164)]; # (linea,r,x,b)
for i in Lista_datos_Lineas:
    L=150
    Linea=i[0].split("-")
    Linea_a=int(Linea[0])-1
    Linea_b=int(Linea[1])-1
    z = complex(i[1],i[2])
    y = complex(0,i[3])
    Zc = cmath.sqrt(z/y)
    gama = cmath.sqrt(z*y)
    Z_modelo=Zc*gama*L
    Y_Serie=1/Z_modelo

    Y_Shunt=gama*L/(Zc*2)
    Y_Shunt=Y_Shunt.imag*1j
    
    A=Y_Serie+Y_Shunt
    B=-Y_Serie
    C=B
    D=A

    coordenadas = [(Linea_a, Linea_a), (Linea_a, Linea_b), 
                   (Linea_b, Linea_a), (Linea_b, Linea_b)]
    valores_a_sumar = [A, B, C, D]
    for coord, valor in zip(coordenadas, valores_a_sumar):
        fila, columna = coord
        Ybuss[fila, columna] += valor

############################Trafos######################################

Lista_datos_Trafos= [("4-5",0,0.1,0),
                     ("4-6",0,0.1,0)]; # (linea,r,x,b)
for i in Lista_datos_Trafos:
    Linea=i[0].split("-")
    Linea_a=int(Linea[0])-1
    Linea_b=int(Linea[1])-1

    Z_trafo=complex(i[1],i[2])
    Y_trafo=1/Z_trafo
    A=Y_trafo
    B=-A
    C=B
    D=A

    valores_a_sumar = [A, B, C, D]
    coordenadas = [(Linea_a, Linea_a), (Linea_a, Linea_b), (Linea_b, Linea_a), (Linea_b, Linea_b)]
    for coord, valor in zip(coordenadas, valores_a_sumar):
        fila, columna = coord
        Ybuss[fila, columna] += valor

############################Generación######################################

Lista_datos_Trafos= [("1",0,1.1,0),
                     ("3",0,1.3,0)]; # (linea,r,x,b)
for i in Lista_datos_Trafos:
    Linea=i[0].split("-")
    Linea_a=int(Linea[0])-1
    Z_trafo=complex(i[1],i[2])
    Y_trafo=1/Z_trafo

    A=Y_trafo
    coordenadas = [(Linea_a, Linea_a)]
    valores_a_sumar = [A]
    for coord, valor in zip(coordenadas, valores_a_sumar):
        fila, columna = coord
        Ybuss[fila, columna] += valor
print(Ybuss)

In [13]:
# ------------- SISTEMA - CERTAMEN 2 ------------- 
import pandapower as pp
net = pp.create_empty_network()

#Barras
b1 = pp.create_bus(net, vn_kv=0.22, name="Barra 1")
b2 = pp.create_bus(net, vn_kv=0.22, name="Barra 2")
b3 = pp.create_bus(net, vn_kv=0.22, name="Barra 3")
b4 = pp.create_bus(net, vn_kv=0.22, name="Barra 4")
b5 = pp.create_bus(net, vn_kv=0.22, name="Barra 5")
b6 = pp.create_bus(net, vn_kv=0.22, name="Barra 6")
b7 = pp.create_bus(net, vn_kv=0.22, name="Barra 7")
b8 = pp.create_bus(net, vn_kv=0.22, name="Barra 8")

#Elementos De Barra
pp.create_ext_grid(net, bus=b1, vm_pu=1.00, name="Fuente Externa") #FUENTE
pp.create_load(net, bus=b3, p_mw=5E-8, q_mvar=0, name="Carga 3")     #CARGAS
pp.create_load(net, bus=b4, p_mw=8.75E-8, q_mvar=0, name="Carga 4")
pp.create_load(net, bus=b5, p_mw=3.75E-8, q_mvar=0, name="Carga 5")
pp.create_load(net, bus=b6, p_mw=3E-8, q_mvar=0, name="Carga 6")
pp.create_load(net, bus=b7, p_mw=1.2E-7, q_mvar=0, name="Carga 7")
pp.create_load(net, bus=b8, p_mw=2.5E-8, q_mvar=0, name="Carga 8")

#Elementos De Rama
tid = pp.create_transformer_from_parameters(net, hv_bus=b2, lv_bus=b1, name="Trafo", sn_mva=0.1,
                                            vn_hv_kv=0.22, vn_lv_kv=0.015, vk_percent=4,vkr_percent=1.0794, 
                                            pfe_kw=1.18, i0_percent=0.1873, shift_grado=0)
#L2-3
pp.create_line(net, from_bus=b2, to_bus=b3, length_km=9, name="Línea 2-3(A)",std_type="NAYY 4x50 SE") 
pp.create_line(net, from_bus=b2, to_bus=b3, length_km=9, name="Línea 2-3(B)",std_type="NAYY 4x50 SE")
#L3-4
pp.create_line(net, from_bus=b3, to_bus=b4, length_km=15, name="Línea 3-4(A)",std_type="NAYY 4x50 SE") 
pp.create_line(net, from_bus=b3, to_bus=b4, length_km=15, name="Línea 3-4(B)",std_type="NAYY 4x50 SE")
#L4-5
pp.create_line(net, from_bus=b4, to_bus=b5, length_km=20, name="Línea 4-5(A)",std_type="NAYY 4x50 SE") 
pp.create_line(net, from_bus=b4, to_bus=b5, length_km=20, name="Línea 4-5(B)",std_type="NAYY 4x50 SE")
#L5-6
pp.create_line(net, from_bus=b5, to_bus=b6, length_km=10, name="Línea 5-6(A)",std_type="NAYY 4x50 SE") 
pp.create_line(net, from_bus=b5, to_bus=b6, length_km=10, name="Línea 5-6(B)",std_type="NAYY 4x50 SE")
#L6-7
pp.create_line(net, from_bus=b6, to_bus=b7, length_km=5, name="Línea 6-7(A)",std_type="NAYY 4x50 SE") 
pp.create_line(net, from_bus=b6, to_bus=b7, length_km=5, name="Línea 6-7(B)",std_type="NAYY 4x50 SE")
#L7-8
pp.create_line(net, from_bus=b7, to_bus=b8, length_km=30, name="Línea 7-8(A)",std_type="NAYY 4x50 SE") 
pp.create_line(net, from_bus=b7, to_bus=b8, length_km=30, name="Línea 7-8(B)",std_type="NAYY 4x50 SE") 
#L8-2
pp.create_line(net, from_bus=b8, to_bus=b2, length_km=10, name="Línea 8-2(A)",std_type="NAYY 4x50 SE") 
pp.create_line(net, from_bus=b8, to_bus=b2, length_km=10, name="Línea 8-2(B)",std_type="NAYY 4x50 SE") 


13